## Import

In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn 
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.model_selection import train_test_split

## Khám phá dữ liệu

In [90]:
data_df = pd.read_csv('Data/train.csv', index_col = 0) # Cho cột index là cột 0 
data_df.head()

,State Name,State Abbreviation,Wind Direction,Created,Applicable Date,Min Temp,Max Temp,The Temp,Wind Speed,Air Pressure,Humidity,Visibility,Predictability
ID,,,,,,,,,,,,,
4.815610e+15,Clear,c,24.918324,2016-01-01T15:50:46.783160Z,1/1/2016,22.5975,33.8400,32.39,5.850634,1015.0,57.0,11.968541,68
5.117610e+15,Light Cloud,lc,111.222444,2016-01-02T15:50:46.519900Z,1/2/2016,24.2150,33.9675,32.26,4.975470,1014.0,61.0,11.687371,70
5.845770e+15,Light Rain,lr,54.357487,2016-01-03T15:50:46.178120Z,1/3/2016,24.6425,33.0950,32.36,5.751001,1014.0,66.0,9.814558,75
4.887680e+15,Heavy Cloud,hc,105.343582,2016-01-04T15:50:46.861790Z,1/4/2016,24.6750,33.3150,32.92,8.274074,1014.0,63.0,11.006659,71
6.722350e+15,Light Cloud,lc,93.195824,2016-01-05T15:50:46.201860Z,1/5/2016,24.3425,33.3975,33.14,8.083090,1013.0,63.0,11.890248,70


### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [91]:
# Kích thước của dữ liệu
data_df.shape

(1022, 13)

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Quan sát bộ dữ liệu ta thấy mỗi dòng chứa thông tin thời tiết của mỗi ngày

### Dữ liệu có các dòng bị lặp không?

In [92]:
# Kiểm tra dữ liệu thu thập được có bị trùng không
data_df.index.duplicated().sum()

3

### Mỗi cột có ý nghĩa gì?

Xem file "description.txt"

In [93]:
with open('description.txt', 'r',encoding = 'utf-8') as f:
    print(f.read())

***** DESCRIPTION WEATHER *****
Vị trí: Hồ Chí Minh City
Thời tiết 4 năm (2016,2017,2018,2019)
Predictability:
+ 80: Thunder
+ 77: Heavy Rain
+ 75: Light Rain
+ 73: Showers
+ 71: Heavy Cloud
+ 70: Light Cloud
+ 68: Clear

ID: id của mỗi ngày trong từng năm
State Name: tên trạng thái của ngày (Clear,Light Cloud,...)
State Abbreviation: viết tắt của trạng thái (c,lc,...)
Wind Direction: Hướng gió
Created: Thời gian cụ thể trong ngày
Applicable Date: Ngày áp dụng
Min Temp: Nhiệt độ tối thiểu
Max Temp: Nhiệt đố tối đa
The Temp: Nhiệt độ
Wind Speed: Tốc độ của gió
Air Pressure: Áp suất không khí
Humidity: Độ ẩm
Visibility: Khoảng cách nhìn thấy






## Đưa ra câu hỏi cần trả lời

Với bộ dữ liệu này thì cột output ta sẽ lấy là cột Predictability  
Output - *(80,77,75,73,71,70,68)* ứng với thời tiết được tính từ input - *các chỉ số trong ngày* - được tính từ công thức nào ?   
Tạm thời ta để input là các chỉ số trong ngày, ta sẽ xử lý các thuộc tính ở bước tiền xử lý  
  

Việc tìm ra câu trả lời sẽ giúp cho mọi người biết được thời tiết như thế nào để có thể chuẩn bị tốt cho một ngày (thực tế trong cuộc sống )  
VD: Nếu biết trời mưa, khi đi ra ngoài ta sẽ cầm theo dù (ô), giúp cho các bác ngư dân tránh tại nạn ,...

## Khám phá dữ liệu (để biết cách tách các tập)

Để biết cách tách các tập thì ta cần khám phá thêm cột output một ít

In [94]:
# Cột output hiện có kiểu dữ liệu gì?
data_df['Predictability'].dtype

dtype('int64')

In [95]:
# Cột output có giá trị thiếu không?
data_df['Predictability'].isna().sum()

0

In [96]:
# Tỉ lệ các lớp trong cột output?
data_df['Predictability'].value_counts(normalize=True) * 100

77    34.148728
75    27.592955
70    14.285714
71    12.524462
73     9.589041
80     1.369863
68     0.489237
Name: Predictability, dtype: float64

Ok, như vậy không có vấn đề gì cả

## Tiền xử lý (tách các tập)

Bây giờ ta sẽ thực hiện bước tiền xử lý là tách tập validation

In [97]:
# Tách X và y
y_sr = data_df["Predictability"] # sr là viết tắt của series
X_df = data_df.drop("Predictability", axis=1)

In [103]:
# Tách tập huấn luyện và tập validation theo tỉ lệ 70%:30%
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3, 
                                                              stratify=y_sr, random_state=0)

In [99]:
train_X_df.shape

(715, 12)

In [62]:
train_y_sr.shape

(715,)

In [63]:
val_X_df.shape

(307, 12)

In [64]:
val_y_sr.shape

(307,)

In [65]:
train_X_df.head().index

Float64Index([5172400000000000.0, 6220460000000000.0, 6174120000000000.0,
              5816520000000000.0, 5498080000000000.0],
             dtype='float64', name='ID')

## Khám phá dữ liệu (tập huấn luyện)

### Mỗi cột input hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

In [66]:
train_X_df.dtypes

State Name             object
State Abbreviation     object
Wind Direction        float64
Created                object
Applicable Date        object
Min Temp              float64
Max Temp              float64
The Temp              float64
Wind Speed            float64
Air Pressure          float64
Humidity              float64
Visibility            float64
dtype: object

### Với mỗi cột input có kiểu dữ liệu dạng số, các giá trị được phân bố như thế nào?

In [67]:
train_X_df.dtypes[train_X_df.dtypes != object]

Wind Direction    float64
Min Temp          float64
Max Temp          float64
The Temp          float64
Wind Speed        float64
Air Pressure      float64
Humidity          float64
Visibility        float64
dtype: object

In [68]:
num_cols = ['Wind Direction','Min Temp','Max Temp','The Temp','Wind Speed','Air Pressure','Humidity','Visibility']
df = train_X_df[num_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def lower_quartile(df):
    return df.quantile(0.25).round(1)
def median(df):
    return df.quantile(0.5).round(1)
def upper_quartile(df):
    return df.quantile(0.75).round(1)
df.agg([missing_ratio, 'min', lower_quartile, median, upper_quartile, 'max'])

,Wind Direction,Min Temp,Max Temp,The Temp,Wind Speed,Air Pressure,Humidity,Visibility
missing_ratio,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.0,0.000000
min,1.943751,19.472500,25.398,24.77,1.342581,996.445,46.0,3.564496
lower_quartile,112.000000,24.100000,30.000,28.60,3.600000,1012.100,65.0,7.500000
median,169.900000,24.800000,31.000,29.80,5.100000,1013.600,77.0,9.700000
upper_quartile,241.000000,25.700000,32.700,31.50,7.200000,1015.000,83.0,11.500000
max,359.171590,28.203333,37.820,36.66,13.850926,1020.510,96.0,18.473366


Theo như bảng ở trên, ta thấy không có thuôc tính (dạng số) bị thiếu dữ liệu

### Với mỗi cột input có kiểu dữ liệu không phải dạng số, các giá trị được phân bố như thế nào?

In [69]:
pd.set_option('display.max_colwidth', 200) # Để nhìn rõ hơn
cat_cols = list(set(train_X_df.columns) - set(num_cols))
df = train_X_df[cat_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def num_values(df):
    return df.nunique()
def value_ratios(c):
    return dict((c.value_counts(normalize=True) * 100).round(1))
df.agg([missing_ratio, num_values, value_ratios])

,State Abbreviation,Created,Applicable Date,State Name
missing_ratio,0,0,0,0
num_values,7,715,715,7
value_ratios,"{'hr': 34.1, 'lr': 27.6, 'lc': 14.3, 'hc': 12.6, 's': 9.7, 't': 1.4, 'c': 0.4}","{'2016-02-16T17:25:47.921170Z': 0.1, '2016-12-22T17:14:55.583400Z': 0.1, '2016-11-13T17:08:47.958540Z': 0.1, '2017-10-10T14:59:46.488800Z': 0.1, '2016-06-01T15:38:47.669160Z': 0.1, '2017-06-22T14:...","{'9/7/2017': 0.1, '11/16/2017': 0.1, '12/29/2017': 0.1, '4/13/2018': 0.1, '9/29/2018': 0.1, '6/19/2018': 0.1, '3/22/2017': 0.1, '1/19/2018': 0.1, '7/22/2018': 0.1, '2/25/2017': 0.1, '8/7/2017': 0....","{'Heavy Rain': 34.1, 'Light Rain': 27.6, 'Light Cloud': 14.3, 'Heavy Cloud': 12.6, 'Showers': 9.7, 'Thunder': 1.4, 'Clear': 0.4}"


Tương tự như thuộc tính không phải dạng số (categorical) cũng không bị thiếu dữ liệu

## Tiền xử lý (tập huấn luyện) 

Đầu tiên ta sẽ bỏ một số cột sau:  
- Bỏ cột 'ID' , thay vào đó ta sẽ để cột 'Applicable Date' làm cột index  
- Bỏ cột 'State Name' và cột 'State Abbreviation' vì đây là 2 cột tương tự giống với cột output  
- Bỏ cột 'Created' vì cột này không có nhiều tác dụng trong việc đánh giá mô hình

In [104]:
class ColDropper(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        # YOUR CODE HERE
        df_transform = X_df.copy()
        df_transform.set_index('Applicable Date',inplace = True)
        df_transform.drop(['State Name','State Abbreviation','Created'],axis = 1,inplace = True)
        return df_transform

In [105]:
col_dropper = ColDropper()
col_dropper.fit(train_X_df)
col_dropper.transform(train_X_df)

,Wind Direction,Min Temp,Max Temp,The Temp,Wind Speed,Air Pressure,Humidity,Visibility
Applicable Date,,,,,,,,
10/14/2018,104.964330,25.770000,30.353333,29.710,2.323313,1016.820,82.0,11.004484
4/14/2017,121.265912,25.108000,35.202000,32.270,6.920914,1015.250,53.0,12.604825
1/6/2016,97.178108,24.080000,33.957500,33.790,8.236321,1013.000,60.0,11.473308
10/16/2018,140.668908,25.893333,30.353333,29.690,4.651437,1016.610,79.0,6.856210
8/31/2017,226.249291,25.962500,31.495000,30.610,3.278653,1014.230,81.0,11.418938
...,...,...,...,...,...,...,...,...
1/19/2018,210.581340,23.210000,28.027500,27.685,2.375434,1014.190,84.0,7.785781
5/30/2017,204.466575,25.798000,31.298000,29.120,3.319812,1014.005,83.0,10.409832
9/28/2016,243.753547,25.010000,29.802500,27.815,4.881548,1012.350,85.0,9.683759


Như ta thấy ở trên thì toàn bộ thuộc tính chỉ còn lại thuộc tính dạng số (numerical): 'Wind Direction', 'Min Temp', 'Max Temp', 'The Temp', 'Wind Speed', 'Air Pressure', 'Humidity', 'Visibility'  
Mặc dù không có dữ liệu bị thiếu trên các thuộc tính dạng kiểu số nhưng ta cũng cần phải ta đều cần tính mean, vì ta không biết được cột nào sẽ bị thiếu giá trị khi dự đoán với các véc-tơ input mới.